In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
import keras
from keras import losses, optimizers, metrics

Using TensorFlow backend.


In [2]:
class ProteinLigandDataGenerator(keras.utils.Sequence):
    def __init__(self, csv_path, batch_size, shuffle=True, dim=(24,24,24), n_channels=4):
        self.df = self.prepare_df(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = None
        self.dim = dim
        self.n_channels = n_channels
        self.on_epoch_end()
    
    def prepare_df(self, csv_path):
        # oversampling happens here, 
        match_count = 1
        random_count = 4
        supplement = random_count - match_count
        
        df = pd.read_csv(csv_path)
        same_idx = df[df['pro_id'] == df['lig_id']]
        duplicates = [same_idx]*supplement
        return df.append(duplicates, ignore_index=True)
        
    
    def __len__(self):
        '''
        Batches per epoch
        '''
        return int(len(self.df.index)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.generate_data(indexes)
        
        return X, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df.index))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def generate_data(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        sub_df = self.df.iloc[indexes]
        sub_df.reset_index(inplace=True)
        for row in sub_df.itertuples():
            idx = row[0]
            full_X = np.load(row.dests)
            X[idx,:,:,:,:] = full_X
            y[idx] = row.score
            
        return X, y

In [3]:
train_generator = ProteinLigandDataGenerator('./data/csv/train_lig_2_pro_pairs.csv', 12)
test_generator = ProteinLigandDataGenerator('./data/csv/test_lig_2_pro_pairs.csv', 12)

In [4]:
model = Sequential()
model.add(Conv3D(96, kernel_size=3, activation='relu', input_shape=(24,24,24,4)))
model.add(BatchNormalization())
model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool3D())
model.add(Conv3D(196, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv3D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling3D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(), metrics=[metrics.mse])

In [ ]:
real_epochs = 4
iterations = real_epochs * len(train_generator)

validation_frequency = 400
epochs = int(iterations/validation_frequency)

histories = []

for i in range(epochs):
    print(i)
    history = model.fit_generator(train_generator, epochs=1, 
                                  steps_per_epoch=validation_frequency,
                                  use_multiprocessing=True, workers=4, 
                                  verbose=1, validation_data=test_generator)
    histories.append(history)
    model.save(f'models/final_epochs_{i}.h5')

0
Epoch 1/1
400/400 [==============================] - 71s 178ms/step - loss: 0.0651 - mean_squared_error: 0.0651 - val_loss: 0.0454 - val_mean_squared_error: 0.0454
1
Epoch 1/1
400/400 [==============================] - 70s 176ms/step - loss: 0.0268 - mean_squared_error: 0.0268 - val_loss: 0.0417 - val_mean_squared_error: 0.0417
Epoch 1/1
2
Epoch 1/1
400/400 [==============================] - 70s 176ms/step - loss: 0.0263 - mean_squared_error: 0.0263 - val_loss: 0.0147 - val_mean_squared_error: 0.0147
3
Epoch 1/1
400/400 [==============================] - 72s 179ms/step - loss: 0.0224 - mean_squared_error: 0.0224 - val_loss: 0.0261 - val_mean_squared_error: 0.0261
4
Epoch 1/1
400/400 [==============================] - 70s 174ms/step - loss: 0.0195 - mean_squared_error: 0.0195 - val_loss: 0.0136 - val_mean_squared_error: 0.0136
5
Epoch 1/1
400/400 [==============================] - 69s 173ms/step - loss: 0.0201 - mean_squared_error: 0.0201 - val_loss: 0.0129 - val_mean_squared_error: 0